In [118]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [119]:
COCO_JSON_DATA = "data/example.json"
OUTPUT_DIR = "output"

# Load dataset

In [120]:
from pycocotools.coco import COCO
coco_data = COCO(COCO_JSON_DATA)

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [121]:
image_file_dict_id2name = {image_data["id"] : image_data["file_name"] for image_data in coco_data.imgs.values()}
print(image_file_dict_id2name)

{1: 'image1.jpg', 2: 'image2.jpg'}


In [122]:
category_dict_id2label = {category_data["id"] : category_data["name"] for category_data in coco_data.cats.values()}
print(category_dict_id2label)

{1: 'label1'}


In [123]:
import BboxToolkit
from dataclasses import dataclass
from typing import List


@dataclass
class OrientedBBox:
    category_name: str
    category_id: int
    x: float
    y: float
    w: float
    h: float
    theta: float  # [rad]

    def __str__(self):
        return f"x:{self.x}, y:{self.y}, w:{self.w}, h:{self.h}, theta:{self.theta * 180 / np.pi}[deg]"

    @property
    def xywht(self):
        return self.x, self.y, self.w, self.h, self.theta

    @property
    def xywht_as_str(self):
        return f"{self.x}, {self.y}, {self.w}, {self.h}, {self.theta}"

    @property
    def polypoints_as_str(self):
        polypoints = BboxToolkit.obb2poly(self.xywht)
        poly_str = f"{polypoints[0]}"
        for value in polypoints[1:]:
            poly_str += f" {value}"
        return poly_str


In [124]:
import BboxToolkit
import numpy as np
from typing import List, Dict

""" Assign empty list to corresponding image
"""
dict_image_name_to_obbs: Dict[str, List[OrientedBBox]] = {}
for image_file_name in image_file_dict_id2name.values():
    dict_image_name_to_obbs[image_file_name] = []


for annotation_data in coco_data.anns.values():
    """Load image and annotation data"""
    image_id = annotation_data["image_id"]
    image_name = image_file_dict_id2name[image_id]
    category_id = annotation_data["category_id"]

    """ Load and convert oriented bounding-box information
    """
    assert len(annotation_data["segmentation"]) == 1

    obb_polygon = annotation_data["segmentation"][0]  # obb: oriented bbox, quadruple pairs of (x_i, y_i)
    assert len(obb_polygon) == 8

    obb_ann_info = BboxToolkit.poly2obb(np.asarray(obb_polygon))
    obb = OrientedBBox(
        category_name=category_dict_id2label[category_id],
        category_id=category_id,
        x=obb_ann_info[0],
        y=obb_ann_info[1],
        w=obb_ann_info[2],
        h=obb_ann_info[3],
        theta=obb_ann_info[4],
    )
    ann_id = annotation_data["id"]
    print(f"{ann_id}:\n xywht:{obb.xywht_as_str}\n poly:{obb.polypoints_as_str}")

    dict_image_name_to_obbs[image_name].append(obb)

1:
 xywht:1530.134521484375, 1766.5614013671875, 1035.64990234375, 78.77658081054688, 1.0999574661254883
 poly:1729.9351806640625 1287.214111328125 1800.1400146484375 1322.949951171875 1330.3338623046875 2245.90869140625 1260.1290283203125 2210.1728515625


# Output data

In [125]:
import os
from pathlib import Path

output_dir_pathlib = Path(OUTPUT_DIR)

""" Assign empty annotation text to corresponding key
"""
dict_image_name_to_annotation_text = {}
for image_name in dict_image_name_to_obbs.keys():
    dict_image_name_to_annotation_text[image_name] = ""

""" Generate annotation text data to output
"""
for image_name, obb_list in dict_image_name_to_obbs.items():
    for obb in obb_list:
        annotation_str = f"{obb.polypoints_as_str} {obb.category_name} 0"
        dict_image_name_to_annotation_text[image_name] += annotation_str
print(dict_image_name_to_annotation_text)

""" Dump
"""
for image_name, annotation_text_data in dict_image_name_to_annotation_text.items():
    image_name_stem = output_dir_pathlib.joinpath(image_name).stem
    output_text_file_pathlib = Path(OUTPUT_DIR, f"{image_name_stem}.txt")
    if output_text_file_pathlib.exists():
        os.remove(str(output_text_file_pathlib))
    with open(str(output_text_file_pathlib), "w") as f:
        f.write(annotation_text_data)

{'image1.jpg': '1729.9351806640625 1287.214111328125 1800.1400146484375 1322.949951171875 1330.3338623046875 2245.90869140625 1260.1290283203125 2210.1728515625 label1 0', 'image2.jpg': ''}
